<a href="https://colab.research.google.com/github/kingamirasadi/cv/blob/main/linear_regression_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import torch
from torch import nn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import torch.nn.functional as F





In [4]:

# fetch dataset
infrared_thermography_temperature = fetch_ucirepo(id=925)

# data (as pandas dataframes)
X = infrared_thermography_temperature.data.features
y = infrared_thermography_temperature.data.targets


In [5]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [6]:
X['Distance'] = X['Distance'].fillna(X['Distance'].mean())

<ipython-input-6-7bc11b51e899>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Distance'] = X['Distance'].fillna(X['Distance'].mean())


In [7]:
y.columns

Index(['aveOralF', 'aveOralM'], dtype='object')

In [8]:
y

,aveOralF,aveOralM
0,36.85,36.59
1,37.00,37.19
2,37.20,37.34
3,36.85,37.09
4,36.80,37.04
...,...,...
1015,36.95,36.99
1016,37.25,37.19
1017,37.35,37.59
1018,37.15,37.29


In [9]:
y = y.drop('aveOralM', axis=1)


In [10]:
y.isnull().sum()

aveOralF    0
dtype: int64

In [11]:
X.head(10)

,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHCC1,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1
0,Male,41-50,White,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,...,33.5775,33.4775,33.3725,33.4925,33.0025,34.5300,34.0075,35.6925,35.6350,35.6525
1,Female,31-40,Black or African-American,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,...,34.0325,34.0550,33.6775,33.9700,34.0025,34.6825,34.6600,35.1750,35.0925,35.1075
2,Female,21-30,White,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,...,34.9000,34.8275,34.6475,34.8200,34.6700,35.3450,35.2225,35.9125,35.8600,35.8850
3,Female,21-30,Black or African-American,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,...,34.4400,34.4225,34.6550,34.3025,34.9175,35.6025,35.3150,35.7200,34.9650,34.9825
4,Male,18-20,White,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,...,35.0900,35.1600,34.3975,34.6700,33.8275,35.4175,35.3725,35.8950,35.5875,35.6175
5,Female,21-30,White,24.0,26.0,0.8,0.8275,35.1325,35.2025,34.5275,...,34.1925,34.2825,34.4800,34.2850,34.2425,34.8600,34.6925,35.8500,35.8175,35.8500
6,Male,21-30,White,24.0,31.0,0.8,0.4425,35.2825,35.6150,34.7500,...,33.4775,33.6025,34.0150,33.6775,34.2775,35.2000,34.6300,35.8375,35.6650,35.7075
7,Female,18-20,White,25.0,30.0,0.8,0.5875,35.1050,34.8700,34.2725,...,33.7275,33.4025,33.3925,33.4600,33.5550,34.1325,33.9975,35.6800,35.4425,35.4900
8,Female,18-20,Asian,25.0,30.0,0.8,0.9775,35.3075,35.4225,34.5575,...,35.1475,35.0250,35.1725,35.1900,35.2900,35.6550,35.4750,35.8525,35.3575,35.3825
9,Male,18-20,Multiracial,26.0,31.0,0.8,1.0675,35.8900,35.7650,35.4225,...,35.2550,35.3525,35.3750,35.2150,35.0100,35.6350,35.4875,36.2050,35.9750,36.0200


In [12]:
encoder = LabelEncoder()

In [13]:
X['Gender'] = encoder.fit_transform(X['Gender'])

<ipython-input-13-3342336525d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gender'] = encoder.fit_transform(X['Gender'])


In [14]:
X['Age'].unique()

array(['41-50', '31-40', '21-30', '18-20', '26-30', '21-25', '>60',
       '51-60'], dtype=object)

In [15]:
age_dict = { '18-20' : 0 ,'21-25' :1 ,'21-30' :2 ,'26-30' :3,'31-40' :4  ,'41-50' :5 ,'51-60' : 6 , '>60' : 7 }

In [ ]:
X['Age'] = X['Age'].map(age_dict)

In [17]:
X.head(5)

,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHCC1,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1
0,1,5,White,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,...,33.5775,33.4775,33.3725,33.4925,33.0025,34.5300,34.0075,35.6925,35.6350,35.6525
1,0,4,Black or African-American,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,...,34.0325,34.0550,33.6775,33.9700,34.0025,34.6825,34.6600,35.1750,35.0925,35.1075
2,0,2,White,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,...,34.9000,34.8275,34.6475,34.8200,34.6700,35.3450,35.2225,35.9125,35.8600,35.8850
3,0,2,Black or African-American,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,...,34.4400,34.4225,34.6550,34.3025,34.9175,35.6025,35.3150,35.7200,34.9650,34.9825
4,1,0,White,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,...,35.0900,35.1600,34.3975,34.6700,33.8275,35.4175,35.3725,35.8950,35.5875,35.6175


In [18]:
X = X.drop('Ethnicity' , axis =1 )

In [19]:
X.isnull().sum()

Gender         0
Age            0
T_atm          0
Humidity       0
Distance       0
T_offset1      0
Max1R13_1      0
Max1L13_1      0
aveAllR13_1    0
aveAllL13_1    0
T_RC1          0
T_RC_Dry1      0
T_RC_Wet1      0
T_RC_Max1      0
T_LC1          0
T_LC_Dry1      0
T_LC_Wet1      0
T_LC_Max1      0
RCC1           0
LCC1           0
canthiMax1     0
canthi4Max1    0
T_FHCC1        0
T_FHRC1        0
T_FHLC1        0
T_FHBC1        0
T_FHTC1        0
T_FH_Max1      0
T_FHC_Max1     0
T_Max1         0
T_OR1          0
T_OR_Max1      0
dtype: int64

In [20]:
# lets dig into Dimensionality Reduction with PCA

In [21]:
col = X.columns

In [22]:
col

Index(['Gender', 'Age', 'T_atm', 'Humidity', 'Distance', 'T_offset1',
       'Max1R13_1', 'Max1L13_1', 'aveAllR13_1', 'aveAllL13_1', 'T_RC1',
       'T_RC_Dry1', 'T_RC_Wet1', 'T_RC_Max1', 'T_LC1', 'T_LC_Dry1',
       'T_LC_Wet1', 'T_LC_Max1', 'RCC1', 'LCC1', 'canthiMax1', 'canthi4Max1',
       'T_FHCC1', 'T_FHRC1', 'T_FHLC1', 'T_FHBC1', 'T_FHTC1', 'T_FH_Max1',
       'T_FHC_Max1', 'T_Max1', 'T_OR1', 'T_OR_Max1'],
      dtype='object')

In [23]:
mms = MinMaxScaler()

In [24]:
scaled_f = mms.fit_transform(X)

In [25]:
New_X = pd.DataFrame(data = scaled_f  , columns= ['Gender', 'Age', 'T_atm', 'Humidity', 'Distance', 'T_offset1',
       'Max1R13_1', 'Max1L13_1', 'aveAllR13_1', 'aveAllL13_1', 'T_RC1',
       'T_RC_Dry1', 'T_RC_Wet1', 'T_RC_Max1', 'T_LC1', 'T_LC_Dry1',
       'T_LC_Wet1', 'T_LC_Max1', 'RCC1', 'LCC1', 'canthiMax1', 'canthi4Max1',
       'T_FHCC1', 'T_FHRC1', 'T_FHLC1', 'T_FHBC1', 'T_FHTC1', 'T_FH_Max1',
       'T_FHC_Max1', 'T_Max1', 'T_OR1', 'T_OR_Max1'])

In [26]:
scaled_target =  mms.fit_transform(y)

In [27]:
y.columns

Index(['aveOralF'], dtype='object')

In [28]:
New_y =pd.DataFrame(data = scaled_target ,columns=['aveOralF'])

In [29]:
std = StandardScaler()

In [30]:
New_X = std.fit_transform(New_X)
New_y = std.fit_transform(New_y)

In [31]:
New_X = pd.DataFrame(data = New_X  , columns= ['Gender', 'Age', 'T_atm', 'Humidity', 'Distance', 'T_offset1',
       'Max1R13_1', 'Max1L13_1', 'aveAllR13_1', 'aveAllL13_1', 'T_RC1',
       'T_RC_Dry1', 'T_RC_Wet1', 'T_RC_Max1', 'T_LC1', 'T_LC_Dry1',
       'T_LC_Wet1', 'T_LC_Max1', 'RCC1', 'LCC1', 'canthiMax1', 'canthi4Max1',
       'T_FHCC1', 'T_FHRC1', 'T_FHLC1', 'T_FHBC1', 'T_FHTC1', 'T_FH_Max1',
       'T_FHC_Max1', 'T_Max1', 'T_OR1', 'T_OR_Max1'])

In [32]:
New_y =pd.DataFrame(data = New_y ,columns=['aveOralF'])

In [33]:
pca = PCA(n_components=2)  # Choose the number of desired components
pca.fit(New_X)
X_reduced = pca.transform(X)



In [34]:
X_R_N = pd.DataFrame(data=X_reduced)

In [35]:
X_R_N.shape

(1020, 2)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_R_N, New_y, test_size=0.2, random_state=42)


In [37]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [38]:
X_tensor = torch.from_numpy(X_train)


In [42]:
class LinearRegressionModel(nn.Module):
  def __init__(self, input_dim):  # input_dim: number of features
    super().__init__()
    self.weights = nn.Parameter(torch.randn(input_dim, 1, requires_grad=True, dtype=torch.float))
    self.bias = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

  def forward(self, x: torch.Tensor):  # x: input features
    return torch.mm(x, self.weights) + self.bias  # Matrix multiplication


In [45]:
X_train.shape[1]

2

In [46]:
model = LinearRegressionModel(X_train.shape[1])  # input_dim based on number of features in X

In [47]:
loss_fn = nn.MSELoss()  # loss Function

In [48]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adjust learning rate as needed

In [ ]:
for epoch in range(100):
  # Forward pass
  y_pred = model(X)

  # Calculate loss
  loss = loss_fn(y_pred, y)

  # Clear gradients
  optimizer.zero_grad()

  # Backpropagation
  loss.backward()

  # Update weights and bias
  optimizer.step()

  # Print loss (optional)
  if epoch % 10 == 0:
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


In [51]:
git remote add origin https://github.com/kingamirasadi/sklearn-pytorch.git

SyntaxError: invalid syntax (<ipython-input-51-e396f4e777c6>, line 1)